In [ ]:
from math import pi, cos, sin, sqrt
import sys
sys.path.append('/root/alok/github/cv_research/alok')

from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

from pyobb.pyobb.obb import OBB
from biomass_utils.points_of_interest import get_data, get_depth_cutoff, get_points_of_interest, distance_between_points

In [ ]:
data_dir_base = '/root/data/blender_data/batch_v1'
idx = 0

data = get_data(data_dir_base, idx)
point_cloud, mask, annotation, image = data['point_cloud'], data['mask'], data['annotation_data'], data['image']


In [ ]:
fish_point_cloud = point_cloud[mask > 0]
depth_histogram = np.histogram(fish_point_cloud[:,1])
cutoff_idx = np.where(depth_histogram[0]==0)[0][0]
depth_cutoff = depth_histogram[1][cutoff_idx]
corrected_fish_point_cloud = fish_point_cloud[fish_point_cloud[:,1] < depth_cutoff]

In [ ]:
obb = OBB.build_from_points([(p[0], p[1], p[2]) for p in corrected_fish_point_cloud])

In [ ]:
obb.centroid

In [ ]:
obb.points

In [ ]:
distance_between_points(obb.points[0], obb.points[6])

In [ ]:
113. / 104

In [ ]:
np.save('/root/alok/point_cloud.npy', corrected_fish_point_cloud)

In [ ]:
annotation

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(point_cloud[:,:,1])

In [ ]:
Image.open('{}/stereo_images/left_0.png'.format(data_dir_base))

In [ ]:
Image.open('{}/stereo_images/right_0.png'.format(data_dir_base))

In [ ]:

# creates a lat/lon sphere with a given radius and centered at a given point
def sphere(radius, center, num_slices=30):
    theta_step = 2.0 * pi / (num_slices - 1)
    phi_step = pi / (num_slices - 1.0)
    theta = 0.0
    vertices = []
    for i in range(0, num_slices):
        cos_theta = cos(theta)
        sin_theta = sin(theta)
        phi = 0.0
        for j in range(0, num_slices):
            x = -sin(phi) * cos_theta
            y = -cos(phi)
            z = -sin(phi) * sin_theta
            n = sqrt(x * x + y * y + z * z)
            if n < 0.99 or n > 1.01:
                x /= n
                y /= n
                z /= n
            vertices.append((x * radius + center[0],
                             y * radius + center[1],
                             z * radius + center[2]))
            phi += phi_step
        theta += theta_step
    return vertices

obb = OBB.build_from_points(sphere(1, (0, 0, 0)))